### Napari tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os , sys
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent

from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
import napari
import zarr
import shutil

In [4]:
viewer = napari.Viewer()

2025-01-28 10:59:41.248 python[90545:13679475] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-28 10:59:41.248 python[90545:13679475] +[IMKInputSession subclass]: chose IMKInputSession_Modern


#### Initiate the SAM2 model like you do in Napari

In [6]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
from octron.sam2_octron.helpers.build_sam2_octron import build_sam2_octron

In [7]:
sam2_folder = Path('sam2_octron')
checkpoint = 'sam2.1_hiera_base_plus.pt' # under folder /checkpoints
model_cfg = 'sam2.1/sam2.1_hiera_b+.yaml' # under folder /configs
# ------------------------------------------------------------------------------------
sam2_checkpoint = cur_path / sam2_folder / Path(f'checkpoints/{checkpoint}')
model_cfg = Path(f'configs/{model_cfg}')


predictor, device  = build_sam2_octron(config_file=model_cfg.as_posix(), 
                                                       ckpt_path=sam2_checkpoint.as_posix(), 
                                     )


Support for MPS devices is preliminary. SAM 2 is trained with CUDA and might give numerically different outputs and sometimes degraded performance on MPS. See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion.
Uing device: mps


/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Loaded SAM2VideoPredictor OCTRON
Model image size: 1024


### From napari, after loading video file, extract info 

In [8]:
viewer.dims.set_point(0,0)
current_indices = viewer.dims.current_step
print(current_indices)

(0, 971, 971)


In [9]:
video_data = viewer.layers[0].data   # the whole video

print(f'video shape: {video_data.shape}')   
num_frames, height, width, channels = video_data.shape


video shape: (1603, 1944, 1944, 3)


In [10]:
def run_new_pred(inference_state,
                frame_idx,
                obj_id, 
                label,
                point):
    assert label in [0,1]
    # Run prediction
    #obj_id : give a unique id to each object we interact with (it can be any integers)
    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
                                                inference_state=inference_state,
                                                frame_idx=frame_idx,
                                                obj_id=obj_id,
                                                points=np.array([point],dtype=np.float32),
                                                labels=np.array([label], np.int32)
                                                )
    
    # Add the mask image as a new labels layer
    mask = (out_mask_logits[0] > 0).cpu().numpy().astype(np.uint8)
    current_label = obj_id+1
    if len(np.unique(mask))>1:
        mask[mask==np.unique(mask)[1]] = current_label 
    mask = mask.squeeze()
    return mask 

In [11]:
inference_state = predictor.init_state(napari_viewer=viewer, video_layer_idx=0)

/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/torch/nn/functional.py:4594: UserWarning: The operator 'aten::upsample_bicubic2d.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn.upsample_bicubic2d(


Initialized SAM2 model


In [12]:
predictor.reset_state(inference_state)

In [13]:

# Instantiate the mask and annotation layers 
# Keep them empty at start 
mask_layer_dummy = np.zeros((num_frames, height, width), dtype=np.uint8)
mask_layer_dummy.shape

colors = cmr.take_cmap_colors('cmr.tropical', 8, cmap_range=(0, 1),
                     return_fmt='int')
colors_norm = np.stack(colors) / 255.0  
cyclic_map = napari.utils.CyclicLabelColormap(
    np.hstack([np.stack(colors_norm), np.ones((len(colors), 1))])
)
labels_layer = viewer.add_labels(
    mask_layer_dummy, 
    name='Mask',  # Name of the layer
    opacity=0.4,  # Optional: opacity of the labels
    colormap=cyclic_map,
    blending='additive'  # Optional: blending mode
)
# Add the points layer to the viewer
points_layer = viewer.add_points(None, 
                                 ndim=3,
                                 name='Annotations', 
                                 scale=(1,1),
                                 size=40,
                                 border_color='dimgrey',
                                 border_width=.2,
                                 opacity=.6,
                                 )
# Store the initial length of the points data
previous_length = len(points_layer.data)


left_right_click = 'left'
def on_mouse_press(layer, event):
    '''
    Generic function to catch left and right mouse clicks
    '''
    global left_right_click
    if event.type == 'mouse_press':
        if event.button == 1:  # Left-click
            left_right_click = 'left'
        elif event.button == 2:  # Right-click
            left_right_click = 'right'


def on_points_added(event):

    global points_layer
    global labels_layer
    global left_right_click
    
    global previous_length
    global inference_state
    
    current_length = len(points_layer.data)
    if current_length > previous_length:
        previous_length = current_length 

        # # Execute prediction 
        newest_point_data =  points_layer.data[-1]
        print(f'{left_right_click} click at {newest_point_data}')
        if left_right_click == 'left':
            label = 1
            points_layer.face_color[-1] = [0.59607846, 0.98431373, 0.59607846, 1.]
            points_layer.symbol[-1] = 'o'
        elif left_right_click == 'right':
            label = 0
            points_layer.face_color[-1] = [1., 1., 1., 1.]
            points_layer.symbol[-1] = 'x'
        points_layer.refresh() 
        # Run prediction
        frame_idx  = int(newest_point_data[0])
        point_data = newest_point_data[1:][::-1]
        print(f"New point added on frame {frame_idx} at {point_data}")  
        mask = run_new_pred(inference_state,
                            frame_idx=frame_idx,
                            obj_id=0,
                            label=label,
                            point=point_data,
                            )
        
        labels_layer.data[frame_idx,:,:]= mask
        labels_layer.refresh()   

points_layer.mouse_drag_callbacks.append(on_mouse_press)
points_layer.events.data.connect(on_points_added)

# Hide the transform, delete, and select buttons
qctrl = viewer.window.qt_viewer.controls.widgets[points_layer]
getattr(qctrl, 'transform_button').setVisible(False)
getattr(qctrl, 'delete_button').setVisible(False)
getattr(qctrl, 'select_button').setVisible(False)
# Select the current, add tool for the points layer
viewer.layers.selection.active = points_layer
viewer.layers.selection.active.mode = 'add'

/var/folders/01/t95cmymn5cj5mmyw65m26t4w0000gn/T/ipykernel_90545/1617221865.py:89: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.6.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  qctrl = viewer.window.qt_viewer.controls.widgets[points_layer]


left click at [  49.          570.05264464 1557.68424759]
New point added on frame 49 at [1557.68424759  570.05264464]


Traceback (most recent call last):
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/backends/_qt.py", line 506, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/napari/_vispy/canvas.py", l

right click at [  49.          570.05264464 1557.68424759]
New point added on frame 49 at [1557.68424759  570.05264464]


Traceback (most recent call last):
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/backends/_qt.py", line 506, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/napari/_vispy/canvas.py", l

In [13]:
predictor.remove_object(inference_state, 0)

([], [])

In [14]:
from napari.qt.threading import thread_worker


In [15]:
obj_id = 0

@thread_worker
def thread_predict(inference_state, frame_idx, max_imgs):
    global labels_layer
    video_segments = {} 
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state, 0, max_imgs):
        video_segments[out_frame_idx] = {
                                            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                                            for i, out_obj_id in enumerate(out_obj_ids)
                                        }
        #  Add the mask image as a new labels layer
        mask = video_segments[out_frame_idx][obj_id] # THIS NEEDS TO BE MADE LAYER SPECIFIC 
        current_label = obj_id+1
        if len(np.unique(mask))>1:
            mask[mask==np.unique(mask)[1]] = current_label 
        mask = mask.squeeze()
        labels_layer.data[out_frame_idx,:,:]= mask
        labels_layer.refresh()   
        viewer.dims.set_point(0,out_frame_idx)
        

In [ ]:
worker = thread_predict(inference_state, frame_idx=0 , max_imgs=200)  # create "worker" object
#worker.returned.connect(viewer.add_image)  # connect callback functions
worker.start()  # start the thread!

In [21]:
# Zarr experiments 

start_idx = 0   
stop_idx = 200
batch_size = 20


# Create temp output dir 
sample_dir = cur_path / 'sample_data'
sample_dir.mkdir(exist_ok=True)
sample_data_zarr = sample_dir / 'sample_data.zarr'
if sample_data_zarr.exists():
    shutil.rmtree(sample_data_zarr)

store = zarr.storage.LocalStore(sample_data_zarr, read_only=False)
root = zarr.create_group(store=store)

# # Create a subgroup for the current object ID
# annotation_group = root.create_group(ann_name)


image_zarr = root.create_array(name='masks',
                                shape=(num_frames, 3, predictor.image_size, predictor.image_size),  
                                chunks=(batch_size, 3, predictor.image_size, predictor.image_size),  
                                dtype='float16')



In [30]:
from torchvision.transforms import Resize

@thread_worker
def resize_to_zarr(image_zarr, start_idx, stop_idx, batch_size):   
    resize_img = Resize(
                    size=(predictor.image_size), # This is 1024x1024 for the l model
                    )
    for idx in range(start_idx, stop_idx, batch_size):
        print(f'Processing frames {idx} to {idx+batch_size}')
        collected_imgs = []
        for img_idx in range(idx, idx+batch_size):   
            curr_img = torch.from_numpy(video_data[img_idx]).permute(2, 0, 1)
            curr_img = resize_img(curr_img)
            curr_img = curr_img.float() / 255.0
            collected_imgs.append(curr_img)
        image_zarr[idx:idx+batch_size] = torch.stack(collected_imgs).numpy()
        
worker = resize_to_zarr(image_zarr, start_idx, stop_idx, batch_size)  
worker.start()  # start the thread!
        

Processing frames 0 to 20


Processing frames 20 to 40
Processing frames 40 to 60
Processing frames 60 to 80
Processing frames 80 to 100
Processing frames 100 to 120
Processing frames 120 to 140
Processing frames 140 to 160
Processing frames 160 to 180
Processing frames 180 to 200


In [31]:
for image in image_zarr[:]:
    image = torch.from_numpy(image)

In [33]:
image.shape

torch.Size([3, 1024, 1024])

left click at [  19.          516.76316966 1433.34213931]
New point added on frame 19 at [1433.34213931  516.76316966]


Traceback (most recent call last):
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/backends/_qt.py", line 506, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/horst/miniconda3/envs/sam2/lib/python3.11/site-packages/napari/_vispy/canvas.py", l

In [ ]:
#  inference_state["output_dict_per_obj"][obj_idx] = {
#                 "cond_frame_outputs": {},  # dict containing {frame_idx: <out>}
#                 "non_cond_frame_outputs": {},  # dict containing {frame_idx: <out>}
#             }

In [ ]:
# output_dict_per_obj is huge 
# Structure
# -> obj_id
# --> cond_frame_outputs
# --> non_cond_frame_outputs

In [ ]:
inference_state["output_dict_per_obj"][obj_idx]['cond_frame_outputs'][0]['obj_ptr'].shape